# 환경 설정

In [2]:
import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [7]:
# 지역 키워드를 입력하기
keyword = '횟집'

# page 정하기
page = 1

In [28]:
# 크롬창 띄우기
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [29]:

# 광고창 없애기

# 광고창 iframe 접근하기
driver.switch_to.frame("google_ads_iframe_/395211568/init/desktop_all_0")

# '다시 보지 않기' 버튼 클릭하기
element = '#ad > div > button.ad_btn.ad_block_btn'
driver.find_element(By.CSS_SELECTOR,element).click()


In [31]:
# 첫 번째 음식점 클릭
element = '/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img'
driver.find_element(By.CSS_SELECTOR,element).click()

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B378B3+2193587]
	Ordinal0 [0x00AD0681+1771137]
	Ordinal0 [0x009E41A8+803240]
	Ordinal0 [0x009E6BB4+814004]
	Ordinal0 [0x009E6A72+813682]
	Ordinal0 [0x009E6D00+814336]
	Ordinal0 [0x00A121B5+991669]
	Ordinal0 [0x00A1273B+993083]
	Ordinal0 [0x00A3F7C2+1177538]
	Ordinal0 [0x00A2D7F4+1103860]
	Ordinal0 [0x00A3DAE2+1170146]
	Ordinal0 [0x00A2D5C6+1103302]
	Ordinal0 [0x00A077E0+948192]
	Ordinal0 [0x00A086E6+952038]
	GetHandleVerifier [0x00DE0CB2+2738370]
	GetHandleVerifier [0x00DD21B8+2678216]
	GetHandleVerifier [0x00BC17AA+512954]
	GetHandleVerifier [0x00BC0856+509030]
	Ordinal0 [0x00AD743B+1799227]
	Ordinal0 [0x00ADBB68+1817448]
	Ordinal0 [0x00ADBC55+1817685]
	Ordinal0 [0x00AE5230+1856048]
	BaseThreadInitThunk [0x76BEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F475F4+228]
	RtlGetAppContainerNamedObjectPath [0x76F4